# Creamos 1 millón de documentos en colección llamada rendimiento

In [3]:
#Conexión a la bbdd y selección de colección
from pymongo import MongoClient
# ojo con la IP del servidor de base de datos. Comprobar en el servicio docker mongodb
cliente = MongoClient('172.18.0.4', 27017, 
                      username='admin', password='abc123...', authSource='admin'
                     )

bbdd = cliente.pruebas
coleccion = bbdd.rendimiento

In [4]:
#Este bloque genera 1MM de documentos y los inserta en la bbdd
from random import seed
from random import random
# seed random number generator
seed(1)

for i in range(1000000):
    coleccion.insert_one({'IDCliente': i, 'Numero': random()*100})

In [5]:
#Comprobamos los índices actuales
coleccion.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'Numero_1': {'v': 2, 'key': [('Numero', 1)]}}

# Comprobamos tiempos sin índice

In [6]:
%%time
#Comprobamos tiempo para recuperar un rango de valores
documentos = coleccion.find({ 'Numero': { '$gt': 3, '$lt': 3.001 } })
for documento in documentos:
    print(documento)

{'_id': ObjectId('66111bbe1dc58d656b060e77'), 'IDCliente': 454293, 'Numero': 3.0000430638567166}
{'_id': ObjectId('6626a4e8e3d42c082e56d623'), 'IDCliente': 454293, 'Numero': 3.0000430638567166}
{'_id': ObjectId('66111b471dc58d656b0134d7'), 'IDCliente': 136437, 'Numero': 3.00030810359605}
{'_id': ObjectId('6626a464e3d42c082e51fc83'), 'IDCliente': 136437, 'Numero': 3.00030810359605}
{'_id': ObjectId('66111c8d1dc58d656b0e6207'), 'IDCliente': 999973, 'Numero': 3.000563002967005}
{'_id': ObjectId('6626a5d0e3d42c082e5f29b3'), 'IDCliente': 999973, 'Numero': 3.000563002967005}
{'_id': ObjectId('66111b6a1dc58d656b02a2d9'), 'IDCliente': 230135, 'Numero': 3.000583044638483}
{'_id': ObjectId('6626a48de3d42c082e536a85'), 'IDCliente': 230135, 'Numero': 3.000583044638483}
{'_id': ObjectId('66111c721dc58d656b0d49ae'), 'IDCliente': 928204, 'Numero': 3.000602301747324}
{'_id': ObjectId('6626a5b1e3d42c082e5e115a'), 'IDCliente': 928204, 'Numero': 3.000602301747324}
{'_id': ObjectId('66111c501dc58d656b0be2

In [7]:
%%time
#Medimos el tiempo para obtener el documento con el valor mínimp
documentos = coleccion.find().sort('Numero',1).limit(1)
for documento in documentos:
    print(documento)

{'_id': ObjectId('66110ead70d17b930fc1dc0f'), 'IDCliente': 0, 'Número': 13.436424411240122}
CPU times: user 4.8 ms, sys: 428 µs, total: 5.23 ms
Wall time: 5.47 ms


# Tiempos tras creación de índice por campo Número

In [8]:
#Comprobamos los índices existentes ahora
coleccion.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'Numero_1': {'v': 2, 'key': [('Numero', 1)]}}

In [9]:
%%time
#Comprobamos tiempo para recuperar un rango de valores
documentos = coleccion.find({ 'Numero': { '$gt': 3, '$lt': 3.001 } })
for documento in documentos:
    print(documento)

{'_id': ObjectId('66111bbe1dc58d656b060e77'), 'IDCliente': 454293, 'Numero': 3.0000430638567166}
{'_id': ObjectId('6626a4e8e3d42c082e56d623'), 'IDCliente': 454293, 'Numero': 3.0000430638567166}
{'_id': ObjectId('66111b471dc58d656b0134d7'), 'IDCliente': 136437, 'Numero': 3.00030810359605}
{'_id': ObjectId('6626a464e3d42c082e51fc83'), 'IDCliente': 136437, 'Numero': 3.00030810359605}
{'_id': ObjectId('66111c8d1dc58d656b0e6207'), 'IDCliente': 999973, 'Numero': 3.000563002967005}
{'_id': ObjectId('6626a5d0e3d42c082e5f29b3'), 'IDCliente': 999973, 'Numero': 3.000563002967005}
{'_id': ObjectId('66111b6a1dc58d656b02a2d9'), 'IDCliente': 230135, 'Numero': 3.000583044638483}
{'_id': ObjectId('6626a48de3d42c082e536a85'), 'IDCliente': 230135, 'Numero': 3.000583044638483}
{'_id': ObjectId('66111c721dc58d656b0d49ae'), 'IDCliente': 928204, 'Numero': 3.000602301747324}
{'_id': ObjectId('6626a5b1e3d42c082e5e115a'), 'IDCliente': 928204, 'Numero': 3.000602301747324}
{'_id': ObjectId('66111c501dc58d656b0be2

In [10]:
%%time
#Medimos el tiempo para obtener el documento con el valor mínimp
documentos = coleccion.find().sort('Numero',1).limit(1)
for documento in documentos:
    print(documento)

{'_id': ObjectId('66110ead70d17b930fc1dc0f'), 'IDCliente': 0, 'Número': 13.436424411240122}
CPU times: user 2.19 ms, sys: 808 µs, total: 3 ms
Wall time: 2.12 ms


# Análisis de planes de ejecución
En el aula lo hacemos utilizando  la consola de Mongo, pero se indican aquí los comandos equivalentes.

In [11]:
# Eliminamos el índice
coleccion.drop_index('Numero_1')

In [12]:
#Obtenemos el plan de ejecución
coleccion.find({ 'Numero': { '$gt': 3, '$lt': 3.001 } }).explain()

{'explainVersion': '1',
 'queryPlanner': {'namespace': 'pruebas.rendimiento',
  'indexFilterSet': False,
  'parsedQuery': {'$and': [{'Numero': {'$lt': 3.001}},
    {'Numero': {'$gt': 3}}]},
  'queryHash': '2A7AE8E8',
  'planCacheKey': '2A7AE8E8',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'COLLSCAN',
   'filter': {'$and': [{'Numero': {'$lt': 3.001}}, {'Numero': {'$gt': 3}}]},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 12,
  'executionTimeMillis': 3340,
  'totalKeysExamined': 0,
  'totalDocsExamined': 4037930,
  'executionStages': {'stage': 'COLLSCAN',
   'filter': {'$and': [{'Numero': {'$lt': 3.001}}, {'Numero': {'$gt': 3}}]},
   'nReturned': 12,
   'executionTimeMillisEstimate': 360,
   'works': 4037931,
   'advanced': 12,
   'needTime': 4037918,
   'needYield': 0,
   'saveState': 4037,
   'restoreState': 4037,
  

In [13]:
#Volvemos a crear el índice
coleccion.create_index([('Numero', 1)])

'Numero_1'

In [11]:
#Vemos ahora cómo ha cambiado el plan
coleccion.find({ 'Numero': { '$gt': 3, '$lt': 3.001 } }).explain()

{'explainVersion': '1',
 'queryPlanner': {'namespace': 'pruebas.rendimiento',
  'indexFilterSet': False,
  'parsedQuery': {'$and': [{'Numero': {'$lt': 3.001}},
    {'Numero': {'$gt': 3}}]},
  'queryHash': '2A7AE8E8',
  'planCacheKey': 'FC7C8AC6',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'FETCH',
   'inputStage': {'stage': 'IXSCAN',
    'keyPattern': {'Numero': 1},
    'indexName': 'Numero_1',
    'isMultiKey': False,
    'multiKeyPaths': {'Numero': []},
    'isUnique': False,
    'isSparse': False,
    'isPartial': False,
    'indexVersion': 2,
    'direction': 'forward',
    'indexBounds': {'Numero': ['(3, 3.001)']}}},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 6,
  'executionTimeMillis': 1,
  'totalKeysExamined': 6,
  'totalDocsExamined': 6,
  'executionStages': {'stage': 'FETCH',
   'nReturned': 6,
   'executionTimeMillisEstimate': 1,
  

# Trabajando con Replica-sets

In [17]:
from pymongo import MongoClient

#Conexión a la bbdd
cliente = MongoClient('172.18.0.4', 27017, 
                      username='admin', password='abc123', authSource='admin',
                      replicaset='miReplica',
                      readPreference='secondaryPreferred'
                     )


In [18]:
#Comprobamos que ha descubierto automáticamente el resto de nodos
print(cliente.nodes)

frozenset()


In [19]:
#Hacemos una consulta
bbdd = cliente.tienda
coleccion = bbdd.pedidos
print(coleccion.find_one())

ServerSelectionTimeoutError: No replica set members available for replica set name "miReplica", Timeout: 30s, Topology Description: <TopologyDescription id: 6626a6dce3d42c082e5f29cf, topology_type: ReplicaSetNoPrimary, servers: []>

In [15]:
#Una vez parado el primario, comprobamos los nodos reconocidos
#Comprobamos que ha descubierto automáticamente el resto de nodos
print(cliente.nodes)

frozenset()


In [16]:
#Repetimos la consulta para verificar que podemos seguir trabajando
print(coleccion.find_one())

ServerSelectionTimeoutError: No replica set members available for replica set name "miReplica", Timeout: 30s, Topology Description: <TopologyDescription id: 6611217c1dc58d656b0e6222, topology_type: ReplicaSetNoPrimary, servers: []>

In [17]:
#Y hacemos una inserción para comprobar que hay un nuevo primario
# (si no hubiese primario no podrías modificar)
resultado = coleccion.insert_one({'IDCliente': 34, 'test': 1})
print(resultado.inserted_id)

ServerSelectionTimeoutError: No replica set members available for replica set name "miReplica", Timeout: 30s, Topology Description: <TopologyDescription id: 6611217c1dc58d656b0e6222, topology_type: ReplicaSetNoPrimary, servers: []>

# Sharding

## Conexión a mongos

In [18]:
from pymongo import MongoClient

#Utiliza la IP de tu anfitrión
cliente = MongoClient('mongos', 27017)

#Creamos la instancia para interactuar con la colección de clientes y pedidos
bbdd = cliente.tienda
coleccion_clientes = bbdd.clientes
coleccion_pedidos = bbdd.pedidos

## Inserción de datos de clientes y de pedidos

In [19]:
#Inserción de documentos de clientes
documentos_clientes = [
    {'IDCliente': 1, 'Nombre': "Juan", 'Apellidos': "Fernández"},
    {'IDCliente': 2, 'Nombre': "María", 'Apellidos': "Fernández"},
    {'IDCliente': 3, 'Nombre': "Carolina", 'Apellidos': "Pérez"}
]

resultado = coleccion_clientes.insert_many(documentos_clientes)
print("Se han insertado",len(resultado.inserted_ids),"clientes")

ServerSelectionTimeoutError: mongos:27017: [Errno -2] Name or service not known (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 661121fa1dc58d656b0e6224, topology_type: Unknown, servers: [<ServerDescription ('mongos', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('mongos:27017: [Errno -2] Name or service not known (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [20]:
#Inserción de documentos de pedidos
documentos_pedidos = [
    {'IDPedido': 1, 'IDCliente': 1, 'Importe': 3.24, 'Ciudad': "Vigo"},
    {'IDPedido': 2, 'IDCliente': 1, 'Importe': 8.01, 'Ciudad': "Pontevedra"},
    {'IDPedido': 3, 'IDCliente': 3, 'Importe': 28.12, 'Ciudad': "A Coruña"},
    {'IDPedido': 4, 'IDCliente': 1, 'Importe': 56.78, 'Ciudad': "Vigo"},
    {'IDPedido': 5, 'IDCliente': 2, 'Importe': 0.12, 'Ciudad': "Madrid"},
    {'IDPedido': 6, 'IDCliente': 3, 'Importe': 99.45, 'Ciudad': "Barcelona"},
    {'IDPedido': 7, 'IDCliente': 3, 'Importe': 2.1, 'Ciudad': "Valencia"},
    {'IDPedido': 8, 'IDCliente': 1, 'Importe': 9, 'Ciudad': "Ourense"},
    {'IDPedido': 9, 'IDCliente': 1, 'Importe': 32.56, 'Ciudad': "Lugo"},
    {'IDPedido': 10, 'IDCliente': 3, 'Importe': 5.45, 'Ciudad': "Santiago"},
]

resultado = coleccion_pedidos.insert_many(documentos_pedidos)
print("Se han insertado",len(resultado.inserted_ids),"pedidos")

ServerSelectionTimeoutError: mongos:27017: [Errno -2] Name or service not known (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 661121fa1dc58d656b0e6224, topology_type: Unknown, servers: [<ServerDescription ('mongos', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('mongos:27017: [Errno -2] Name or service not known (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

## Comprobación de datos insertados

In [21]:
#Número de documentos en colección clientes
num_clientes = coleccion_clientes.count_documents({})
print("Hay",num_clientes,"clientes")

#Número de documentos en colección pedidos
num_pedidos = coleccion_pedidos.count_documents({})
print("Hay",num_pedidos,"pedidos")

ServerSelectionTimeoutError: mongos:27017: [Errno -2] Name or service not known (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 661121fa1dc58d656b0e6224, topology_type: Unknown, servers: [<ServerDescription ('mongos', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('mongos:27017: [Errno -2] Name or service not known (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

## Conexión a mongoshard1 y cuenta de documentos

In [ ]:
from pymongo import MongoClient

#Como JupyterLab y los nodos de Mongo están conectados a través de la red interna
# establecemos la conexión a travñes de dicha red. Todos escuchan en esa red en el puerto 27017.
# Si te quisieses conectar a través del anfitrión, debes hacerlo al puerto mapeado, en esta caso
# al 27019
cliente = MongoClient('mongoshard1', 27017)

#Creamos la instancia para interactuar con la colección de clientes y pedidos
bbdd = cliente.tienda
coleccion_clientes = bbdd.clientes
coleccion_pedidos = bbdd.pedidos

#Número de documentos en colección clientes
num_clientes = coleccion_clientes.count_documents({})
print("Hay",num_clientes,"clientes")

#Número de documentos en colección pedidos
num_pedidos = coleccion_pedidos.count_documents({})
print("Hay",num_pedidos,"pedidos")

## Conexión a mongoshard2 y cuenta de documentos

In [ ]:
from pymongo import MongoClient

#Como JupyterLab y los nodos de Mongo están conectados a través de la red interna
# establecemos la conexión a travñes de dicha red. Todos escuchan en esa red en el puerto 27017.
# Si te quisieses conectar a través del anfitrión, debes hacerlo al puerto mapeado, en esta caso
# al 27020
cliente = MongoClient('mongoshard2', 27017)

#Creamos la instancia para interactuar con la colección de clientes y pedidos
bbdd = cliente.tienda
coleccion_clientes = bbdd.clientes
coleccion_pedidos = bbdd.pedidos

#Número de documentos en colección clientes
num_clientes = coleccion_clientes.count_documents({})
print("Hay",num_clientes,"clientes")

#Número de documentos en colección pedidos
num_pedidos = coleccion_pedidos.count_documents({})
print("Hay",num_pedidos,"pedidos")